In [1]:
import os
import torch
import matplotlib.pyplot as plt
from torchvision.models import detection
import numpy as np
import cv2
from skimage.io import imread
from collections import Counter

In [2]:
folder_path = 'bike' #original images path
model_name = 'faster'
w = 1000
h = 300

In [3]:
COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

In [6]:
if model_name == 'faster':
    model = detection.fasterrcnn_resnet50_fpn(pretrained=True)
else:
    model = detection.maskrcnn_resnet50_fpn(pretrained=True)

model.eval()

MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(in

In [7]:
data_list = os.listdir(folder_path)
pred = []

for i in range(len(data_list)):
    img_path = os.path.join(folder_path, data_list[i])

    image = imread(img_path) 
    dim = (w, h)
    image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    image1 = image.transpose((2, 0, 1))    
    image1 = np.expand_dims(image1, axis=0)
    image1 = image1 / 255.0
    image1 = torch.FloatTensor(image1)

    
    predictions = model(image1)
    boxes = predictions[0]['boxes'].detach()
    boxes = boxes.cpu().numpy().astype(np.uint16).tolist()
    scores = predictions[0]['scores'].detach().cpu().numpy().tolist()
    labels = predictions[0]['labels'].detach().cpu().numpy().tolist()
    for label, box,s in zip(labels, boxes, scores):
        if s<0.4:
            continue
        pred.append([i,s,[COCO_INSTANCE_CATEGORY_NAMES[label],box]])
        cv2.rectangle(image, (box[0], box[1]), (box[2], box[3] ), color=(0,255,0), thickness=2)
        pred_class = COCO_INSTANCE_CATEGORY_NAMES[label]
        conf = "{} %".format(round(s,2))
        cv2.putText(image,pred_class, (box[0]-10, box[1]-20), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0,255,0),thickness=1)
        cv2.putText(image,conf, (box[0]-10, box[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0,255,0),thickness=1)
    cv2.imwrite('mask_bike/bike{}.png'.format(i),image)
    

In [ ]:
person=0
car=0
for i in range(len(pred)):
    if pred[i][2][0]=='person':
        person +=1
    if pred[i][2][0]=='car':
        car +=1
    
        

## YOLOV5

In [9]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model.conf=0.4
data_list = os.listdir(folder_path)

for i in range(len(data_list)):
    img_path = os.path.join(folder_path, data_list[i])
    image = imread(img_path) 
    dim = (w, h)
    image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    results = model(image)
    results.save()

Using cache found in /home/sahar/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-2-15 torch 1.8.0+cu111 CPU

Fusing layers... 
Model Summary: 213 layers, 7225885 parameters, 0 gradients, 16.5 GFLOPs
Adding AutoShape... 
Saved 1 image to runs/detect/exp
Saved 1 image to runs/detect/exp2
Saved 1 image to runs/detect/exp3
Saved 1 image to runs/detect/exp4
Saved 1 image to runs/detect/exp5
Saved 1 image to runs/detect/exp6
Saved 1 image to runs/detect/exp7
Saved 1 image to runs/detect/exp8
Saved 1 image to runs/detect/exp9
Saved 1 image to runs/detect/exp10
Saved 1 image to runs/detect/exp11
Saved 1 image to runs/detect/exp12
Saved 1 image to runs/detect/exp13
Saved 1 image to runs/detect/exp14
